# Meeting Data Clean

Meeting Indicator Description:
- eventid: vbrick event id
- start_time: meeting scheduled start time
- end_time: meeting scheduled end time
- meeting_key: meeting key
- site: extract from webexsiteurl
- region: extract from vbricksiterul
- polls_enabled: polls enabled
- chat_enabled: chat enabled
- qa_enabled: qa enabled
- custom_branding_enabled: custom branding enabled
- count_AccountAdmin: count of AccountAdmin
- count_Attendee: count of attendee
- count_Host: count of host
- count_Moderator: count of moderator
- session_time_seconds_sum: transform from sessionTime (format HH:MM:ss), sum of all moderators' session time
- session_time_seconds_mean: transform from sessionTime (format HH:MM:ss), mean of all moderators' session time
- session_time_seconds_median: transform from sessionTime (format HH:MM:ss), median of all moderators' session time
- session_time_seconds_max: transform from sessionTime (format HH:MM:ss), max of all moderators' session time
- session_time_seconds_min: transform from sessionTime (format HH:MM:ss), min of all moderators' session time
- viewing_time_seconds_sum: transform from viewingTime (format HH:MM:ss), sum of all moderators' session time
- viewing_time_seconds_mean: transform from viewingTime (format HH:MM:ss), mean of all moderators' session time
- viewing_time_seconds_median: transform from viewingTime (format HH:MM:ss), median of all moderators' session time
- viewing_time_seconds_max: transform from viewingTime (format HH:MM:ss), max of all moderators' session time
- viewing_time_seconds_min: transform from viewingTime (format HH:MM:ss), min of all moderators' session time

Attendee Indicator Description:
- eventid: vbrick event id
- ip_address: IP address
- browser: browser
- device_type: PC / Mobile
- attendee_type: AccountAdmin / Host / Moderator / Attendee
- org: extract from user email
- session_time_seconds: transform from sessionTime (format HH:MM:ss)
- viewing_time_seconds: transform from viewingTime (format HH:MM:ss)

In [1]:
import numpy as np
import pandas as pd
import re

"""
function list
"""

def convert_time(t):
    arr = t.split(':')
    return float(arr[0]) * 60 * 60 + int(arr[1]) * 60 + int(arr[2])


def extract_org(email):
    return email.split('@')[1]

In [ ]:
meeting_df = pd.read_excel('D:\\workspace\\meeting-info.xlsx', dtype={'start_time': np.datetime64, 'end_time': np.datetime64})
attendee_df = pd.read_excel('D:\\workspace\\attendee-list.xlsx', dtype={'entered_date': np.datetime64, 'exited_date': np.datetime64, 'viewing_start_time': np.datetime64})

# clean meeting data
meeting_df['region'] = meeting_df.apply(lambda row: re.search('https://(.+?).livestream', row.vbrick_site_url).group(1), axis=1)
meeting_df['site'] = meeting_df.apply(lambda row: re.search('(.+?).webex.com', row.webex_site_url).group(1), axis=1)

# clean attendee data
attendee_df['session_time_seconds'] = attendee_df['session_time'].apply(convert_time)
attendee_df['viewing_time_seconds'] = attendee_df['viewing_time'].apply(convert_time)
attendee_df['org'] = attendee_df['email'].apply(extract_org)

In [ ]:

attendee_subset_df = attendee_df[['eventid', 'ip_address', 'browser', 'device_type', 'attendee_type', 'org', 'session_time_seconds', 'viewing_time_seconds']]
meeting_subset_df = meeting_df[['eventid', 'start_time', 'end_time', 'meeting_key', 'site', 'region', 'polls_enabled', 'chat_enabled', 'qa_enabled', 'custom_branding_enabled']]

In [ ]:
# count attendee by type
group_attendee_df = attendee_df.groupby(['eventid', 'attendee_type'], as_index=False)['eventid'].agg(['count'])

attendee_cnt_df = pd.pivot_table(group_attendee_df, index='eventid', columns='attendee_type', fill_value=0)
attendee_cnt_df.columns = ['_'.join(col) for col in attendee_cnt_df.columns.values]

attendee_cnt_df = attendee_cnt_df.reset_index()

# aggregate time relative
group_time_df = attendee_df.groupby('eventid', as_index=False)['session_time_seconds', 'viewing_time_seconds'].agg(['sum', 'mean', 'median', 'max', 'min'])
group_time_df.columns = ['_'.join(col) for col in group_time_df.columns.values]

attendee_time_df = group_time_df
attendee_time_df = group_time_df.reset_index()



In [63]:
meeting_clean_df = pd.merge(meeting_subset_df, attendee_cnt_df, how='outer', on='eventid')
meeting_clean_df = pd.merge(meeting_clean_df, attendee_time_df, how='outer', on='eventid')

# fill 0 for numeric measures
meeting_clean_df['count_AccountAdmin'] = meeting_clean_df['count_AccountAdmin'].fillna(0)
meeting_clean_df['count_Attendee'] = meeting_clean_df['count_Attendee'].fillna(0)
meeting_clean_df['count_Host'] = meeting_clean_df['count_Host'].fillna(0)
meeting_clean_df['count_Moderator'] = meeting_clean_df['count_Moderator'].fillna(0)
meeting_clean_df['session_time_seconds_sum'] = meeting_clean_df['session_time_seconds_sum'].fillna(0)
meeting_clean_df['session_time_seconds_mean'] = meeting_clean_df['session_time_seconds_mean'].fillna(0)
meeting_clean_df['session_time_seconds_median'] = meeting_clean_df['session_time_seconds_median'].fillna(0)
meeting_clean_df['session_time_seconds_max'] = meeting_clean_df['session_time_seconds_max'].fillna(0)
meeting_clean_df['session_time_seconds_min'] = meeting_clean_df['session_time_seconds_min'].fillna(0)
meeting_clean_df['viewing_time_seconds_sum'] = meeting_clean_df['viewing_time_seconds_sum'].fillna(0)
meeting_clean_df['viewing_time_seconds_mean'] = meeting_clean_df['viewing_time_seconds_mean'].fillna(0)
meeting_clean_df['viewing_time_seconds_median'] = meeting_clean_df['viewing_time_seconds_median'].fillna(0)
meeting_clean_df['viewing_time_seconds_max'] = meeting_clean_df['viewing_time_seconds_max'].fillna(0)
meeting_clean_df['viewing_time_seconds_min'] = meeting_clean_df['viewing_time_seconds_min'].fillna(0)

# persis to excel
meeting_clean_df.to_excel('meeting-clean-data.xlsx')
attendee_subset_df.to_excel('attendee-clean-data.xlsx')

In [10]:
a = pd.read_excel('D:\\workspace\\meetinginfo-by-meetinguuid-diff.xlsx')[['eventid', 'livestreamaddr']]
b = pd.read_excel('D:\\workspace\\meeting-clean-data-diff.xlsx')

a['event_started'] = ~a['livestreamaddr'].isna()
a = a[['eventid', 'event_started']]

a.head()

b = b.merge(a, how='inner', on='eventid')
b.head()


,eventid,start_time,end_time,meeting_key,site,region,polls_enabled,chat_enabled,qa_enabled,custom_branding_enabled,...,session_time_seconds_mean,session_time_seconds_median,session_time_seconds_max,session_time_seconds_min,viewing_time_seconds_sum,viewing_time_seconds_mean,viewing_time_seconds_median,viewing_time_seconds_max,viewing_time_seconds_min,event_started
0,da16526f-bb38-477a-82f4-0933898e3094,2021-06-23 18:00:00,2021-06-23 19:00:00,1754671026,merz-events,us,False,False,True,False,...,333.0,333.0,333,333,223,223.0,223.0,223,223,True
1,24ec59bc-3df0-4f69-a786-f1d523453ddf,2021-07-01 08:55:00,2021-07-01 09:55:00,1867089336,acecloud,eu,True,True,True,False,...,0.0,0.0,0,0,0,0.0,0.0,0,0,False
2,4716802a-b962-4301-9308-1da0cf5edb52,2021-06-30 13:30:00,2021-06-30 14:30:00,1751278087,pvhevents,eu,True,True,True,False,...,139.0,139.0,165,113,156,78.0,78.0,156,0,True
3,25a8f5e6-da95-4814-a4f6-7828f91428bf,2021-06-23 09:00:00,2021-06-23 10:00:00,1751787325,merz-events,us,False,False,True,False,...,0.0,0.0,0,0,0,0.0,0.0,0,0,False
4,6219a564-ead8-469d-b7c4-cf739bc4f695,2021-06-15 13:15:00,2021-06-15 14:15:00,1725022609,dal,us,True,True,True,False,...,818.8,835.0,1350,50,2050,410.0,43.0,1180,0,True
